# CLEAN の基本メソッドまとめ

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

---------------------
### ダミーDataFrame(テスト用）

In [3]:
df = pd.DataFrame({
    'A':[1,5,10],
    'B':[2,6,11],
    'C':[3, np.nan,12],
    'D':[4,8,np.nan]
    }
)
df.set_index(['A','B','C','D'])

df2 = pd.DataFrame(
    [
        ['green','M', 10.1, 'class1'],
        ['red', 'L', 13.6, 'class1'],
        ['blue', 'XL', 15.3, 'class2']
    ]
)
df2.columns = ['color', 'size', 'price', 'classlabel']
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.6,class1
2,blue,XL,15.3,class2


### ワインのデータを読み込む（テスト用）

In [18]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',header=None)
df_wine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


----------------------

### NAの処理 ###

In [4]:
# すべての列がNaNである業だけを削除
df.dropna(how='all')
#欠測値を含む行を削除
df.dropna()
#欠測値を含む列を削除
df.dropna(axis=1)
#非NaNが４つ未満の行を削除
df.dropna(thresh=4)
#特定の列にNaNが含まれている行だけを削除
df.dropna(subset=['C'])

,A,B,C,D
0,1,2,3,4
2,10,11,12,NaN


### 欠測値の補完（平均値補完）

In [5]:
# 欠測値補完（平均値補完）のインスタンスを生成
imr = Imputer(missing_values='NaN', strategy = 'mean', axis=0)
# データを適合
imr = imr.fit(df)
# 補完を実行
imputed_data = imr.transform(df.values)

### カテゴリデータの処理

1) 順序特徴量のマッピング

Tシャツのサイズを整数に変換

In [6]:
# 処理前の状態
df2

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.6,class1
2,blue,XL,15.3,class2


In [7]:
size_mapping = {'XL':3, 'L':2, 'M':1}
df2['size'] = df2['size'].map(size_mapping)
df2

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.6,class1
2,blue,3,15.3,class2


2) クラスラベルを整数に変換

In [8]:
# クラスラベルのエンコーディング
class_mapping = {
    label:idx for idx, label in enumerate(np.unique(df2['classlabel']))
}
df2['classlabel'] = df2['classlabel'].map(class_mapping)
df2
# ラベルに戻す場合は以下のようにする
# inv_class_mapping = {v: k for k, v in class_mapping.items()}
# df2['classlabel'] = df2['classlabel'].map(inv_class_mapping)

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.6,0
2,blue,3,15.3,1


3) 特徴量のOne-Hotエンコーディング

In [9]:
### scikit-learnのLabelEncoderを使ってみる
class_le = LabelEncoder()
y = class_le.fit_transform(df2['classlabel'].values)
y
# クラスラベルを元の文字列に戻す
y2=class_le.inverse_transform(y)
y2

X = df2[['color','size','price']].values
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.6],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

-----
### データセットをトレーニングデータセットとテストデータセットに分割する

In [19]:
# 列名を指定
df_wine.columns = ['Class Label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD of diluted wines', 'Proline']
# 先頭五行を表示
print(df_wine.head())
# クラスラベルのパターンを表示
# ３種類のクラスはぶどうの種類を意味している。
np.unique(df_wine['Class Label'])

   Class Label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0            1    14.23        1.71  2.43               15.6        127   
1            1    13.20        1.78  2.14               11.2        100   
2            1    13.16        2.36  2.67               18.6        101   
3            1    14.37        1.95  2.50               16.8        113   
4            1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD of diluted wines  Proline  
0             5.64  1.04                 3.92     1065  
1       

array([1, 2, 3])

1) model_selectionのtrain_test_split関数で分割する

In [20]:
# 特徴量とクラスラベルを別々に抽出
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:,0].values

In [21]:
# 全体の30%をテストデータにする
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

-------------------
## 特徴量の尺度を揃える
### 正規化(normalization)と標準化(standardization)

### 1)  Min-Maxスケーリング

最大値と最小値を使って正規化する方法。

$$ x_{norm}^{(i)} =  \frac{(x - x_{min})}{x_{max} - x_{min}} $$

In [22]:
# min-maxスケーリングのインスタンスを作成
mms = MinMaxScaler()

In [23]:
# トレーニングデータをスケーリング
X_train_norm = mms.fit_transform(X_train)
# テストデータをスケーリング
X_test_norm = mms.fit_transform(X_test)

### 2) 標準化
平均と標準偏差を使った正規化の方法。

$$ x_{std}^{(i)} = \frac{x^{(i)} - \mu_{x}}{\sigma_{x}} $$


In [24]:
stdsc = StandardScaler()

In [25]:
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.fit_transform(X_test)